# Welcome to czapi

> A basic API for scraping curling boxscores off of the popular [CurlingZone](https://www.curlingzone.com) website. 

## Install

```
pip install czapi
```

## How to use

In [1]:
import czapi.api as api

## General Information
The czapi is based around the LinescorePage object which represents a linescore page, shown below:

![Linescore Page](./imgs/game_by_event_draw_game_number.png)

Creating an instance of the LinescorePage class gives access to every boxscore on that linescore page.

In [2]:
linescore_page = api.LinescorePage(cz_event_id = 6400, cz_draw_id = 2)

The boxscores on the linescore page can be accessed through the `boxscores` property which returns a list of boxscores.

In [3]:
linescore_page.boxscores

[defaultdict(list,
             {'Wayne Tuck Jr.': defaultdict(list,
                          {'href': 'event.php?view=Team&eventid=6400&teamid=144353&profileid=12486#1',
                           'hammer': True,
                           'score': ['0',
                            '2',
                            '0',
                            '0',
                            '0',
                            '0',
                            '1',
                            '1',
                            '1',
                            '0'],
                           'finalscore': '5'}),
              'Matthew Hall': defaultdict(list,
                          {'href': 'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435#1',
                           'hammer': False,
                           'score': ['0',
                            '0',
                            '4',
                            '0',
                            '0',
                            

A boxscore is returned as a dictionary of team names, game information dictionary key, value pairs.

Each game information dictionary contains:

* 'href' key with a corresponding string value of CurlingZone IDs identifying the team.
* 'hammer' key with corresponding boolean value of whether or not that team started the game with hammer.
* 'score' key with corresponding list of string value of end-by-end results for that team.
* 'finalscore' key with corresponding final score for the team.

Individual boxscores can be accessed through the `get_boxscore_from` method.

In [4]:
linescore_page.get_boxscore_from(cz_game_id = 1)

defaultdict(list,
            {'Wayne Tuck Jr.': defaultdict(list,
                         {'href': 'event.php?view=Team&eventid=6400&teamid=144353&profileid=12486#1',
                          'hammer': True,
                          'score': ['0',
                           '2',
                           '0',
                           '0',
                           '0',
                           '0',
                           '1',
                           '1',
                           '1',
                           '0'],
                          'finalscore': '5'}),
             'Matthew Hall': defaultdict(list,
                         {'href': 'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435#1',
                          'hammer': False,
                          'score': ['0',
                           '0',
                           '4',
                           '0',
                           '0',
                           '1',
                   

`cz_game_id` argument corresponds to the number the boxscore appears in on the linescore page.

The LinescorePage object contains these properties which details information on the boxscores:

* event_name
* event_date
* draw

In [5]:
print(linescore_page.event_name,',',linescore_page.event_date,',' ,linescore_page.draw)

Ontario Tankard - Open Qualifier , Jan 17 - 19, 2020 , Draw: 2


## About czapi
czapi is a Python library for scraping curling linescores.